In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import zipfile
from getpass import getuser
from datetime import datetime
import pandas as pd
import numpy as np

## 定期檢查

In [142]:
old_time = '2021-10-25 02:41   '
chrome_options=Options()
driver = webdriver.Chrome(chrome_options=chrome_options)
ftp_html = 'https://ftp.ncbi.nih.gov/pub/taxonomy/'
driver.get(ftp_html)
date = driver.find_element_by_xpath('/html/body/pre/a[4]').text
print(date)
soup = BeautifulSoup(driver.page_source, 'html.parser')
target_position = str(soup).find('<a href="new_taxdump/">new_taxdump/</a>') + len('<a href="new_taxdump/">new_taxdump/</a>')
start_position = str(soup)[target_position:].find('2') + target_position
end_position = str(soup)[start_position:].find(' - ') + start_position
print(str(soup)[start_position:end_position])

if str(soup)[start_position:end_position] != old_time :
    print('更新')
    driver.find_element_by_link_text("new_taxdump/").click()
    driver.find_element_by_link_text("new_taxdump.zip").click()

C:\Users\Tingchun.TC.Hung\Anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


new_taxdump/
2021-10-26 20:40   


## 搬移檔案&解壓縮&檔案讀取轉csv

In [3]:
user_name = getuser()
data_name = 'new_taxdump.zip'
Save_path = 'C:/Users/Tingchun.TC.Hung/Database/'

In [ ]:
os.replace('C:/Users/'+user_name+'/Downloads/'+data_name, Save_path+data_name )
with zipfile.ZipFile(Save_path + data_name, 'r') as zip:
    zip.extractall('Database')
    print('File is unzipped in temp folder')

In [4]:
Data_collect = []

with open(Save_path+'rankedlineage.dmp','r',encoding='utf-8') as file :
    times = 0
    for line in file :
        line = line[:-1] +'\t'
        Collect = line.split('\t|\t')
        times+=1
        if Collect[9] == 'Bacteria' :
            Collect.pop(8)
            Data_collect.append(Collect[:-1])

Name_collect = []

with open(Save_path+'names.dmp','r',encoding='utf-8') as file :
    times = 0
    for line in file :
        line = line[:-1] +'\t'
        Collect = line.split('\t|\t')
        times+=1
        Collect.pop(2)
        Name_collect.append(Collect[:-1])

rankedlineage = pd.DataFrame(Data_collect,columns=['tax_id','tax_name','species','genus','family','order','class','phylum','kingdom'])
rankedlineage

taxid_namesdmp = pd.DataFrame(Name_collect,columns=['tax_id','name','other'])
taxid_namesdmp

save_time = datetime.now().strftime('_%Y-%m-%d-%H-%M-%S')

rankedlineage.to_csv('D:/Record/CRC/自動化/rankedlineage'+save_time+'.csv')
taxid_namesdmp.to_csv('D:/Record/CRC/自動化/taxid_namesdmp'+save_time+'.csv')

with open('C:/Users/Tingchun.TC.Hung/Database/'+'save_time.txt','a') as file :
    file.write(save_time)



D = pd.read_csv('D:/Record/CRC/自動化/rankedlineage'+save_time+'.csv')
S = pd.read_csv('D:/Record/CRC/自動化/taxid_namesdmp'+save_time+'.csv',index_col='tax_id')

tax_name = list(D['tax_name'].values)
Level_order = list(D.columns)[2:]

Target_level = 'species'

if len(Level_order) > Level_order.index(Target_level)+1 :
    Top_level = Level_order[Level_order.index(Target_level)+1]
    Next_level_button = True
else:
    Next_level_button = False
    
index = D[Target_level].values
Real_index = []
for each in index :
    if str(each) != 'nan' :
        Real_index.append(each)
list_index = set(Real_index)
list_index = list(list_index)


Finals,Final_tax_id,Final_tax_name,Final_species,Final_genus,Final_family,Final_order,Final_class,Final_phylum,Final_kingdom = [],[],[],[],[],[],[],[],[],[]
not_in_taxid_namesdmp = []
lose_num = 0

for i in range(len(list_index)):
    Final = []
    if tax_name.count(list_index[i]) != 1:
        print(list_index[i])
        appear_positions = np.where(np.array(tax_name)==list_index[i])[0]
        for run_each_appear in appear_positions:
            if Next_level_button :
                if str(D.loc[run_each_appear][Target_level]) == 'nan' and str(D.loc[run_each_appear][Top_level]) != 'nan':
                    print(D.loc[run_each_appear]['tax_id'])
                    Final_tax_id.append(D.loc[tax_name_appear]['tax_id'])
                    Final_tax_name.append(list_index[i])
                    Final_species.append(D.loc[tax_name_appear]['species'])
                    Final_genus.append(D.loc[tax_name_appear]['genus'])
                    Final_family.append(D.loc[tax_name_appear]['family'])
                    Final_order.append(D.loc[tax_name_appear]['order'])
                    Final_class.append(D.loc[tax_name_appear]['class'])
                    Final_phylum.append(D.loc[tax_name_appear]['phylum'])
                    Final_kingdom.append(D.loc[tax_name_appear]['kingdom'])
                    
                    try:
                        Find = S.loc[D.loc[tax_name_appear]['tax_id']]
                        try : 
                            names = Find['name'].values
                            others = Find['other'].values
                            for num in range(len(names)):
                                Final.append(names[num]+'('+others[num]+')')
                        except:
                            Final.append(Find['name']+'('+Find['other']+')')
                    except:
                        Find = []
                        not_in_taxid_namesdmp.append([Final_tax_id.append(D.loc[tax_name_appear]['tax_id']),list_index[i]])
                    Finals.append(Final)
                    break

    else:
        tax_name_appear = tax_name.index(list_index[i])
        Final_tax_id.append(D.loc[tax_name_appear]['tax_id'])
        Final_tax_name.append(list_index[i])
        Final_species.append(D.loc[tax_name_appear]['species'])
        Final_genus.append(D.loc[tax_name_appear]['genus'])
        Final_family.append(D.loc[tax_name_appear]['family'])
        Final_order.append(D.loc[tax_name_appear]['order'])
        Final_class.append(D.loc[tax_name_appear]['class'])
        Final_phylum.append(D.loc[tax_name_appear]['phylum'])
        Final_kingdom.append(D.loc[tax_name_appear]['kingdom'])
        
        try:
            Find = S.loc[D.loc[tax_name_appear]['tax_id']]
            try : 
                names = Find['name'].values
                others = Find['other'].values
                official_name_position = list(others).index('scientific name')
                Final.append(names[official_name_position])
                for num in range(len(names)):
                    if num != official_name_position :
                        Final.append(names[num])
            except:
                if Find['other'] == 'scientific name' :
                    Final.append(Find['name'])
                else:
                    Final.append('')
                    Final.append(Find['name'])
        except:
            Final = [list_index[i]]
            not_in_taxid_namesdmp.append([D.loc[tax_name_appear]['tax_id'],list_index[i]])
        Finals.append(Final)

Final_Bacteria = pd.DataFrame(Finals,index = Final_tax_id)
Columns = ['scientific name']
Columns_extend = [ 'Synonym'+str(i) for i in range(1,Final_Bacteria.shape[1]) ] 
Columns.extend(Columns_extend)
Final_Bacteria = pd.DataFrame(Finals,index = Final_tax_id,columns=Columns)

C:\Users\Tingchun.TC.Hung\Anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,4,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Tingchun.TC.Hung\Anaconda3\envs\NLP\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
Final_Bacteria

,scientific name,Synonym1,Synonym2,Synonym3,Synonym4,Synonym5,Synonym6,Synonym7,Synonym8,Synonym9,...,Synonym207,Synonym208,Synonym209,Synonym210,Synonym211,Synonym212,Synonym213,Synonym214,Synonym215,Synonym216
392734,Terriglobus roseus,Terriglobus roseus Eichorst et al. 2007,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
283822,Snowella litoralis,"Gomphosphaeria litoralis Hayren, 1921",Gomphosphaeria litoralis,Snowella litoralis (Hayren) Komarek et Hindak ...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
309887,Deinococcus maricopensis,Deinococcus maricopensis Rainey and da Costa 2005,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1902831,Candidatus Mastigocoleus perforans,"""Candidatus Mastigocoleus perforans"" Couradeau...",Mastigocoleus perforans,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
685,Vibrio diazotrophicus,Vibrio diazotrophicus Guerinot et al. 1982,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256428,Tolypothrix scytonemoides,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
450471,Microcoleus paludosus,Microcoleus paludosus Gomont ex Gomont 1892,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2782526,Rhizonema andinum,Rhizonema andinum Lucking & Dal Forno,Rhizonema sp. 'cyanobiont of Dictyonema huaorani',None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
380392,Desulfomicrobium thermophilum,Desulfomicrobium thermophilum Thevenieau et al...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [7]:
pd.DataFrame(not_in_taxid_namesdmp) # 未收錄 學名別稱表

""


In [8]:
Final_Bacteria_family = pd.DataFrame([Final_tax_name,Final_species,Final_genus,Final_family,Final_order,Final_class,Final_phylum,Final_kingdom],index=['tax_name','species','genus','family','order','class','phylum','kingdom'],columns=Final_tax_id).T
Final_Bacteria_family

,tax_name,species,genus,family,order,class,phylum,kingdom
392734,Terriglobus roseus,NaN,Terriglobus,Acidobacteriaceae,Acidobacteriales,Acidobacteriia,Acidobacteria,Bacteria
283822,Snowella litoralis,NaN,Snowella,Coelosphaeriaceae,Synechococcales,NaN,Cyanobacteria,Bacteria
309887,Deinococcus maricopensis,NaN,Deinococcus,Deinococcaceae,Deinococcales,Deinococci,Deinococcus-Thermus,Bacteria
1902831,Candidatus Mastigocoleus perforans,NaN,Mastigocoleus,Hapalosiphonaceae,Nostocales,NaN,Cyanobacteria,Bacteria
685,Vibrio diazotrophicus,NaN,Vibrio,Vibrionaceae,Vibrionales,Gammaproteobacteria,Proteobacteria,Bacteria
...,...,...,...,...,...,...,...,...
256428,Tolypothrix scytonemoides,NaN,Tolypothrix,Tolypothrichaceae,Nostocales,NaN,Cyanobacteria,Bacteria
450471,Microcoleus paludosus,NaN,Microcoleus,Microcoleaceae,Oscillatoriales,NaN,Cyanobacteria,Bacteria
2782526,Rhizonema andinum,NaN,Rhizonema,Rhizonemataceae,Nostocales,NaN,Cyanobacteria,Bacteria
380392,Desulfomicrobium thermophilum,NaN,Desulfomicrobium,Desulfomicrobiaceae,Desulfovibrionales,Deltaproteobacteria,Proteobacteria,Bacteria


In [117]:
Final_Bacteria.to_csv('D:/Record/CRC/自動化/Species_name'+save_time+'.csv')
Final_Bacteria_family.to_csv('D:/Record/CRC/自動化/Species_Biological classification'+save_time+'.csv')

## 差異比較

In [170]:
import datacompy, pandas as pd, sys
 
files = []
try:
    files = [sys.argv[1], sys.argv[2]]
except:
    print('命令行未检测到文件参数。')
if len(files) == 0:
    files = ['./TempFiles/temp_77.csv', './TempFiles/temp_77_new.csv']

In [172]:
compare = datacompy.Compare(Final_Bacteria_family, Final_Bacteria_family_old, join_columns='tax_name')
print(compare.matches()) # 最后判断是否相等，返回 bool
print(compare.report()) # 打印报告详情，返回 string

False
DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        8  6011
1       df2        8  6010

Column Summary
--------------

Number of columns in common: 8
Number of columns in df1 but not in df2: 0
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: tax_name
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 6,007
Number of rows in df1 but not in df2: 4
Number of rows in df2 but not in df1: 3

Number of rows with some compared columns unequal: 6
Number of rows with all compared columns equal: 6,001

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 7
Total number of values which compare unequal: 6

Columns with Unequal Values or Types
------------------------------------

    Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null Diff
1   

In [126]:
compare = datacompy.Compare(Final_Bacteria, Final_Bacteria_old, join_columns='scientific name')
print(compare.matches()) # 最后判断是否相等，返回 bool
print(compare.report()) # 打印报告详情，返回 string

False
DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1      217  6010
1       df2       54  5958

Column Summary
--------------

Number of columns in common: 54
Number of columns in df1 but not in df2: 163
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: scientific name
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 5,844
Number of rows in df1 but not in df2: 166
Number of rows in df2 but not in df1: 114

Number of rows with some compared columns unequal: 5,610
Number of rows with all compared columns equal: 234

Column Comparison
-----------------

Number of columns compared with some values unequal: 53
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 17,355

Columns with Unequal Values or Types
------------------------------------

       Column df1 dtype df2 dtype  # Unequal  M

In [106]:
df1 = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9],[7,8,6]])
df1

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9
3,7,8,6


In [107]:
df2 = pd.DataFrame([[1,2,3],[7,8,6]])
df2

,0,1,2
0,1,2,3
1,7,8,6


In [108]:
compare = datacompy.Compare(df1, df2, join_columns='0')
print(compare.matches()) # 最后判断是否相等，返回 bool
print(compare.report()) # 打印报告详情，返回 string

False
DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        3     4
1       df2        3     2

Column Summary
--------------

Number of columns in common: 3
Number of columns in df1 but not in df2: 0
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: 0
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 2
Number of rows in df1 but not in df2: 2
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 1
Number of rows with all compared columns equal: 1

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 2
Total number of values which compare unequal: 1

Columns with Unequal Values or Types
------------------------------------

  Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null Diff
0      2     int64  